In [1]:
# https://www.kaggle.com/kaerunantoka/h-m-how-to-calculate-map-12/notebook

import numpy as np
import pandas as pd
import gc
import os
import time
import random
from tqdm.auto import tqdm
import seaborn as sns

In [3]:
def visualize_df(df):
    print(df.shape)
    display(df.head())

In [4]:
transactions_train = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
visualize_df(transactions_train)

(31788324, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [5]:
sub = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv')
del sub['prediction']; gc.collect()
visualize_df(sub)

(1371980, 1)


,customer_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...


In [6]:
val_start_date = '2020-09-16'

train_data = transactions_train.query(f"t_dat < '{val_start_date}'").reset_index(drop=True)
valid_data = transactions_train.query(f"t_dat >= '{val_start_date}'").reset_index(drop=True)

visualize_df(train_data)
visualize_df(valid_data)

(31548013, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


(240311, 5)


,t_dat,customer_id,article_id,price,sales_channel_id
0,2020-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,786022008,0.048441,2
1,2020-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,913272003,0.032288,2
2,2020-09-16,000fb6e772c5d0023892065e659963da90b1866035558e...,889669006,0.056508,2
3,2020-09-16,0010e8eb18f131e724d6997909af0808adbba057529edb...,237347060,0.033881,1
4,2020-09-16,0010e8eb18f131e724d6997909af0808adbba057529edb...,562245001,0.013542,1


In [7]:
valid_unq = valid_data.groupby('customer_id')['article_id'].apply(list).reset_index()
valid_unq['valid_true'] = valid_unq['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
visualize_df(valid_unq)

(68984, 3)


,customer_id,article_id,valid_true
0,00039306476aaf41a07fed942884f16b30abfa83a2a8be...,[624486001],0624486001
1,0003e867a930d0d6842f923d6ba7c9b77aba33fe2a0fbf...,[827487003],0827487003
2,000493dd9fc463df1acc2081450c9e75ef8e87d5dd17ed...,"[757926001, 788575004, 640021019]",0757926001 0788575004 0640021019
3,000525e3fe01600d717da8423643a8303390a055c578ed...,[874110016],0874110016
4,00077dbd5c4a4991e092e63893ccf29294a9d5c46e8501...,"[903762001, 879189005, 158340001, 867966009, 9...",0903762001 0879189005 0158340001 0867966009 09...


In [9]:
train_unq = train_data.groupby('customer_id')['article_id'].apply(list).reset_index()
train_unq['valid_pred'] = train_unq['article_id'].map(lambda x: '0'+' 0'.join(str(x)[1:-1].split(', ')))
visualize_df(train_unq)

(1356709, 3)


,customer_id,article_id,valid_pred
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[625548001, 176209023, 627759010, 697138006, 5...",0625548001 0176209023 0627759010 0697138006 05...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[583558001, 639677008, 640244003, 521269001, 6...",0583558001 0639677008 0640244003 0521269001 06...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[663713001, 541518023, 663713001, 578020002, 7...",0663713001 0541518023 0663713001 0578020002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[742079001, 732413001]",0742079001 0732413001
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[634249005, 677049001, 698286003, 707704003, 3...",0634249005 0677049001 0698286003 0707704003 03...


In [10]:
merged = pd.merge(sub, train_unq, on='customer_id', how='left').fillna('')
merged = pd.merge(merged, valid_unq, on='customer_id', how='left').fillna('')

del merged['article_id_x'], merged['article_id_y']; gc.collect()
merged.head()

,customer_id,valid_pred,valid_true
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0625548001 0176209023 0627759010 0697138006 05...,
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0583558001 0639677008 0640244003 0521269001 06...,
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001 0541518023 0663713001 0578020002 07...,
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0742079001 0732413001,
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0634249005 0677049001 0698286003 0707704003 03...,


In [17]:
# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/306007
# https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py


def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
            A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
            A list of lists of elements that are to be predicted 
            (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [18]:
tqdm.pandas()

mapk(merged['valid_true'].map(lambda x: x.split()), 
        merged['valid_pred'].map(lambda x: x.split()), k=12)

0.00014681608404665519